In [1]:
import pandas as pd

In [2]:
import os


os.getcwd()

'/Users/dh/Desktop/dacon_rag'

In [4]:
df = pd.read_csv("baseline_submission_qwen_chroma.csv")

In [5]:
df

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,330,000백만원입니다.\n\n..."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업의 목적은 창업기업의 사업화를 지원하...
2,TEST_002,중소벤처기업부의 혁신창업사업화자금(융자) 사업은 중소기업진흥에 관한 법률 제66조...
3,TEST_003,2010년에 신규 지원된 혁신창업사업화자금은 개발기술사업화자금을 내역사업으로 통합...
4,TEST_004,2020년에 신규 지원된 자금은 미래기술육성자금과 고성장촉진자금입니다.\n\n ...
...,...,...
93,TEST_093,재정정책에서 공적보증채무는 공공부문 단위가 계약에 의해 다른 공공부문과 민간부문 ...
94,TEST_094,"미래사회보장급여에 대한 순의무는 계약상의 부채는 아니지만, 미래 발생 비용으로 여..."
95,TEST_095,"국가결산보고서에서는 '충당부채', '우발부채' 및 '우발자산'이라는 용어를 사용하..."
96,TEST_096,"우발부채는 미래의 의제의무를 포함하는 부채로, 재정건전성과 재정위기 관리 등의 이..."


In [6]:
df["Answer"] = df["Answer"].apply(lambda x: x[:x.find("\n")])

In [8]:
df.to_csv("baseline_submission_qwen_chroma.csv", index=False)

# Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

# Download Library

답변은 반드시 한문장으로 작성하세요

In [2]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 41.6 MB/s eta 0:00:0000:0100:01
zsh:1: no matches found: transformers[torch]
  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.7 kB)
Using cached faiss_cpu-1.8.0.post1-cp310-cp310-macosx_11_0_arm64.whl (6.0 MB)


# Import Library

In [36]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
# from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Vector DB

In [37]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="BAAI/bge-m3"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


# DB 생성

In [38]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]/Users/dh/miniforge3/envs/kiwi/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:07<01:00,  7.53s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:12<00:40,  5.85s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [01:16<00:29, 14.76s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [01:29<00:14, 14.23s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [01:46<00:00, 11.83s/it]


# MODEL Import

In [8]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID 
    model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

#     model = PeftModel.from_pretrained(model, "./persona/checkpoint-200",is_trainable=True)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #temperature=0.2,
        return_full_text=False,
        max_new_tokens=450,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

In [9]:
# LLM 파이프라인
llm = setup_llm_pipeline()

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

You are using a model of type exaone to instantiate a model of type gemma2. This is not supported for all configurations of models and can yield errors.


RuntimeError: No GPU found. A GPU is needed for quantization.

# Langchain 을 이용한 추론

In [41]:
pdf_databases.items()

dict_items([('중소벤처기업부_혁신창업사업화자금(융자)', {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06d0190>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06d0190>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}), ('보건복지부_부모급여(영아수당) 지원', {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06d5b10>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06d5b10>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}), ('보건복지부_노인장기요양보험 사업운영', {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06f2a10>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06f2a10>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}), ('산업통상자원부_에너지바우처', {'db': <langc

In [42]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06d0190>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})

In [44]:
# Embedding
model_kwargs = {'device': 'cpu'} # gpu -> 'cuda', cpu -> 'cpu'

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3", 
    model_kwargs=model_kwargs, 
    encode_kwargs=model_kwargs,
    show_progress=True
)

In [45]:
from langchain_community.vectorstores import Chroma
# 저장된 DB를 불러오기
database = Chroma(
    persist_directory="./Semantic",
    embedding_function=embeddings
)

/Users/dh/miniforge3/envs/kiwi/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [46]:
database

In [39]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    print(normalized_keys)
    retriever = normalized_keys[source]['retriever']
    print(retriever)
    break
    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}
    
    주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

{'중소벤처기업부_혁신창업사업화자금(융자)': {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06d0190>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06d0190>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}, '보건복지부_부모급여(영아수당) 지원': {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06d5b10>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06d5b10>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}, '보건복지부_노인장기요양보험 사업운영': {'db': <langchain_community.vectorstores.faiss.FAISS object at 0x2a06f2a10>, 'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2a06f2a10>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})}, '산업통상자원부_에너지바우처': {'db': <langchain_community.vec

# Submission

In [8]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)